# Project Overview / Data Model

## revenue_df Definition
Task 1 — Analytical Dataset Definition (revenue_df)
Purpose

revenue_df is the analytical fact table used for all revenue KPIs.
It contains only valid sales records with reliable Amount values and excludes cancelled or invalid orders to avoid noise in revenue calculations.

Inclusion Rules

A row is included in revenue_df only if:

Qty > 0

Status ≠ Cancelled

Amount is not null (missing Amounts are filled only when reliably estimable)

Exclusion Rules

The following rows are explicitly excluded:

Cancelled orders

Rows with Qty = 0

Rows with missing Amounts that cannot be estimated reliably

Grain

Each row represents one SKU in one order (order line item).

Amount Reliability Rule

Missing Amount values are not filled using mean/mode.
Instead, they are estimated using SKU-based logic (e.g., average unit price per SKU) only when the estimation is reliable.
This ensures data integrity while allowing the dataset to be used for revenue analysis.

In [1]:
# Import necessary libraries for data analysis
import pandas as pd 
import numpy as np

In [2]:
# Load the Amazon Sale Report dataset
df=pd.read_csv(r"C:\Users\yusuf\Desktop\DataScience Journey\non_guided_projects\E‑commerce Sales Dashboard\Amazon Sale Report.csv")

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\3120272242.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\yusuf\Desktop\DataScience Journey\non_guided_projects\E‑commerce Sales Dashboard\Amazon Sale Report.csv")


In [52]:
# Clean column names by stripping leading/trailing whitespace
df.columns = df.columns.str.strip()
print(df.columns.tolist())

['Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-country', 'promotion-ids', 'B2B']


In [3]:
# Display the first few rows of the dataset
df

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,128970,406-6001380-7673107,05-31-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,...,INR,517.00,HYDERABAD,TELANGANA,500013.0,IN,NaN,False,NaN,False
128971,128971,402-9551604-7544318,05-31-22,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,...,INR,999.00,GURUGRAM,HARYANA,122004.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False
128972,128972,407-9547469-3152358,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,...,INR,690.00,HYDERABAD,TELANGANA,500049.0,IN,NaN,False,NaN,False
128973,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,INR,1199.00,Halol,Gujarat,389350.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False


In [4]:
# Display the first 5 rows for a quick overview
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [5]:
df.set_index("index", inplace=True)
df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
index,,,,,,,,,,,,,,,,,,,,,
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [6]:
4# Display information about the DataFrame including data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Order ID            128975 non-null  object 
 1   Date                128975 non-null  object 
 2   Status              128975 non-null  object 
 3   Fulfilment          128975 non-null  object 
 4   Sales Channel       128975 non-null  object 
 5   ship-service-level  128975 non-null  object 
 6   Style               128975 non-null  object 
 7   SKU                 128975 non-null  object 
 8   Category            128975 non-null  object 
 9   Size                128975 non-null  object 
 10  ASIN                128975 non-null  object 
 11  Courier Status      122103 non-null  object 
 12  Qty                 128975 non-null  int64  
 13  currency            121180 non-null  object 
 14  Amount              121180 non-null  float64
 15  ship-city           128942 non-null  ob

In [7]:
# Get the shape of the DataFrame (number of rows and columns)
df.shape

(128975, 23)

In [8]:
# Display the data types of each column in the DataFrame
df.dtypes

Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
Unnamed: 22            object
dtype: object

In [9]:
# Drop unnecessary columns that are not needed for analysis
df.drop(columns=["ASIN","ship-postal-code","Unnamed: 22"],inplace=True)

In [10]:
# Display the data types of each column after dropping columns
df.dtypes

Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
dtype: object

In [11]:
# Convert the Date column to datetime format for proper date handling
df['Date']=pd.to_datetime(df['Date'],format='%m-%d-%y')

In [12]:
# Display data types after converting Date column
df.dtypes

Order ID                      object
Date                  datetime64[ns]
Status                        object
Fulfilment                    object
Sales Channel                 object
ship-service-level            object
Style                         object
SKU                           object
Category                      object
Size                          object
Courier Status                object
Qty                            int64
currency                      object
Amount                       float64
ship-city                     object
ship-state                    object
ship-country                  object
promotion-ids                 object
B2B                             bool
fulfilled-by                  object
dtype: object

In [13]:
# Display the first 5 rows to verify changes
df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-country,promotion-ids,B2B,fulfilled-by
index,,,,,,,,,,,,,,,,,,,,
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,IN,NaN,False,Easy Ship
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN
3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,IN,NaN,False,Easy Ship
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,IN,NaN,False,NaN


In [14]:
# Check for missing values in the DataFrame after initial cleaning
df.isna().sum()

Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
Courier Status         6872
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
dtype: int64

In [15]:
# Get unique values in the Courier Status column
df['Courier Status'].unique()

array([nan, 'Shipped', 'Cancelled', 'Unshipped'], dtype=object)

In [16]:
# Count the occurrences of each unique value in Courier Status
df['Courier Status'].value_counts()

Courier Status
Shipped      109487
Unshipped      6681
Cancelled      5935
Name: count, dtype: int64

In [17]:
# Count the number of missing values in Courier Status
df['Courier Status'].isna().sum()

np.int64(6872)

In [55]:
# Fill missing values in Courier Status with 'Unknown'
df['Courier Status'].fillna('Unknown',inplace=True)

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\4222944596.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Courier Status'].fillna('Unknown',inplace=True)


In [19]:
# Check for missing values after filling Courier Status
df.isna().sum()

Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
Courier Status            0
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
dtype: int64

In [20]:
# Drop rows with missing shipping city, state, or country
df.dropna(subset=["ship-city","ship-state","ship-country"],inplace=True)
# Check for missing values after dropping rows
df.isna().sum()

Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
Courier Status            0
Qty                       0
currency               7793
Amount                 7793
ship-city                 0
ship-state                0
ship-country              0
promotion-ids         49145
B2B                       0
fulfilled-by          89678
dtype: int64

In [21]:
# Get unique values in the currency column
df['currency'].unique()

array(['INR', nan], dtype=object)

In [22]:
# Fill missing values in currency with 'INR'
df['currency'].fillna('INR',inplace=True)
# Check for missing values after filling currency
df.isna().sum()

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\2089410306.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['currency'].fillna('INR',inplace=True)


Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
Courier Status            0
Qty                       0
currency                  0
Amount                 7793
ship-city                 0
ship-state                0
ship-country              0
promotion-ids         49145
B2B                       0
fulfilled-by          89678
dtype: int64

In [23]:
# Get unique values in the fulfilled-by column
df['fulfilled-by'].unique()

array(['Easy Ship', nan], dtype=object)

In [24]:
# Get the current shape of the DataFrame
df.shape

(128942, 20)

In [25]:
# Drop the fulfilled-by column as it may not be needed
df.drop(columns=['fulfilled-by'], inplace=True)

In [26]:
# Display data types after recent changes
df.dtypes

Order ID                      object
Date                  datetime64[ns]
Status                        object
Fulfilment                    object
Sales Channel                 object
ship-service-level            object
Style                         object
SKU                           object
Category                      object
Size                          object
Courier Status                object
Qty                            int64
currency                      object
Amount                       float64
ship-city                     object
ship-state                    object
ship-country                  object
promotion-ids                 object
B2B                             bool
dtype: object

In [27]:
# Check for missing values in the cleaned DataFrame
df.isna().sum()

Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
Courier Status            0
Qty                       0
currency                  0
Amount                 7793
ship-city                 0
ship-state                0
ship-country              0
promotion-ids         49145
B2B                       0
dtype: int64

In [28]:
# Fill missing values in promotion-ids with 'No Promotion Applied'
df['promotion-ids'].fillna('No Promotion Applied',inplace=True)
df.isna().sum()

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\3480385804.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['promotion-ids'].fillna('No Promotion Applied',inplace=True)


Order ID                 0
Date                     0
Status                   0
Fulfilment               0
Sales Channel            0
ship-service-level       0
Style                    0
SKU                      0
Category                 0
Size                     0
Courier Status           0
Qty                      0
currency                 0
Amount                7793
ship-city                0
ship-state               0
ship-country             0
promotion-ids            0
B2B                      0
dtype: int64

In [56]:
# Create a copy of the cleaned DataFrame for further processing
df1=df.copy()

In [58]:
# Define the analytical dataset revenue_df with inclusion rules
valid_revenue_mask = (
    (df1['Qty'] > 0) &
    (df1['Status'] != 'Cancelled')
)
sku_avg_price = (
    df1.loc[
        valid_revenue_mask & df1['Amount'].notna()
    ]
    .groupby('SKU')
    .apply(lambda x: x['Amount'].sum() / x['Qty'].sum())
)
df1.loc[
    valid_revenue_mask & df1['Amount'].isna(),
    'Amount'
] = (
    df1.loc[
        valid_revenue_mask & df1['Amount'].isna(),
        'Qty'
    ] * df1.loc[
        valid_revenue_mask & df1['Amount'].isna(),
        'SKU'
    ].map(sku_avg_price)
)
df1.loc[valid_revenue_mask, 'Amount'].isna().sum()
revenue_df = df1.loc[valid_revenue_mask & df1['Amount'].notna()]

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\110287228.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x['Amount'].sum() / x['Qty'].sum())


In [57]:
# Print the column names of df1 to verify
print(df1.columns.tolist())

['Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-country', 'promotion-ids', 'B2B']


In [31]:
# Get the shape of the revenue_df DataFrame
revenue_df.shape

(110499, 19)

In [32]:
# Display the first 5 rows of revenue_df
revenue_df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-country,promotion-ids,B2B
index,,,,,,,,,,,,,,,,,,,
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,Shipped,1,INR,406.0,BENGALURU,KARNATAKA,IN,Amazon PLCC Free-Financing Universal Merchant ...,False
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,Shipped,1,INR,329.0,NAVI MUMBAI,MAHARASHTRA,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,Shipped,1,INR,574.0,CHENNAI,TAMIL NADU,IN,No Promotion Applied,False
5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,Shipped,1,INR,824.0,GHAZIABAD,UTTAR PRADESH,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False
6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,Shipped,1,INR,653.0,CHANDIGARH,CHANDIGARH,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False


In [33]:
# Check for missing values in revenue_df
revenue_df.isna().sum()

Order ID              0
Date                  0
Status                0
Fulfilment            0
Sales Channel         0
ship-service-level    0
Style                 0
SKU                   0
Category              0
Size                  0
Courier Status        0
Qty                   0
currency              0
Amount                0
ship-city             0
ship-state            0
ship-country          0
promotion-ids         0
B2B                   0
dtype: int64

In [34]:
# Get descriptive statistics for the Amount column in revenue_df
revenue_df['Amount'].describe()

count    110499.000000
mean        649.323565
std         284.058966
min           0.000000
25%         449.000000
50%         606.000000
75%         788.000000
max        5685.000000
Name: Amount, dtype: float64

In [37]:
# Remove duplicate orders based on Order ID
revenue_df.drop_duplicates(subset=['Order ID'],inplace=True)
# Get the shape after removing duplicates
revenue_df.shape

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\2299688538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_df.drop_duplicates(subset=['Order ID'],inplace=True)


(103159, 19)

In [38]:
revenue_df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-country,promotion-ids,B2B
index,,,,,,,,,,,,,,,,,,,
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,Shipped,1,INR,406.0,BENGALURU,KARNATAKA,IN,Amazon PLCC Free-Financing Universal Merchant ...,False
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,Shipped,1,INR,329.0,NAVI MUMBAI,MAHARASHTRA,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,Shipped,1,INR,574.0,CHENNAI,TAMIL NADU,IN,No Promotion Applied,False
5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,Shipped,1,INR,824.0,GHAZIABAD,UTTAR PRADESH,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False
6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,Shipped,1,INR,653.0,CHANDIGARH,CHANDIGARH,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False


In [42]:
revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103159 entries, 1 to 128974
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Order ID            103159 non-null  object        
 1   Date                103159 non-null  datetime64[ns]
 2   Status              103159 non-null  object        
 3   Fulfilment          103159 non-null  object        
 4   Sales Channel       103159 non-null  object        
 5   ship-service-level  103159 non-null  object        
 6   Style               103159 non-null  object        
 7   SKU                 103159 non-null  object        
 8   Category            103159 non-null  object        
 9   Size                103159 non-null  object        
 10  Courier Status      103159 non-null  object        
 11  Qty                 103159 non-null  int64         
 12  currency            103159 non-null  object        
 13  Amount              103159 non-nul

In [60]:
# Get unique values in the Sales Channel column of revenue_df
revenue_df["Sales Channel"].unique()

array(['Amazon.in', 'Non-Amazon'], dtype=object)

In [59]:
# Print the column names of revenue_df
print(revenue_df.columns.tolist())

['Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-country', 'promotion-ids', 'B2B']


In [ ]:
# Create Order Month column as period
revenue_df['Order Month'] = revenue_df['Date'].dt.to_period('M')
# Create Order Month Number column
revenue_df['Order Month Number'] = revenue_df['Date'].dt.month
# Create Order Year column
revenue_df['Order Year'] = revenue_df['Date'].dt.year
# Display the first 5 rows to verify new columns
revenue_df.head()

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\3386205727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_df['Order Month'] = revenue_df['Date'].dt.to_period('M')
C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\3386205727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_df['Order Month Number'] = revenue_df['Date'].dt.month
C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\3386205727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,currency,Amount,ship-city,ship-state,ship-country,promotion-ids,B2B,Order Month,Order Month Number,Order Year
index,,,,,,,,,,,,,,,,,,,,,
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,INR,406.0,BENGALURU,KARNATAKA,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,2022-04,4,2022
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,INR,329.0,NAVI MUMBAI,MAHARASHTRA,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,2022-04,4,2022
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,...,INR,574.0,CHENNAI,TAMIL NADU,IN,No Promotion Applied,False,2022-04,4,2022
5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,...,INR,824.0,GHAZIABAD,UTTAR PRADESH,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,2022-04,4,2022
6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,...,INR,653.0,CHANDIGARH,CHANDIGARH,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,2022-04,4,2022


In [ ]:
# Create Promo Flag column based on promotion-ids
revenue_df['Promo Flag'] = np.where(revenue_df['promotion-ids'] == 'No Promotion Applied', 'No', 'Yes')
# Count the values in Promo Flag column
revenue_df['Promo Flag'].value_counts()

C:\Users\yusuf\AppData\Local\Temp\ipykernel_28628\1418130776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_df['Promo Flag'] = np.where(revenue_df['promotion-ids'] == 'No Promotion Applied', 'No', 'Yes')


Promo Flag
Yes    79432
No     31067
Name: count, dtype: int64

In [ ]:
# QA Check 1: Ensure no cancelled orders in revenue_df
revenue_df[revenue_df['Status'].str.lower() == 'cancelled'].shape

(0, 23)

In [ ]:
# QA Check 2: Ensure no rows with Qty equal to 0 in revenue_df
revenue_df[revenue_df['Qty'] == 0].shape

(0, 23)

In [ ]:
# QA Check 3: Ensure no missing values in Amount column in revenue_df
revenue_df['Amount'].isna().sum()

np.int64(0)

In [ ]:
# QA Check 4: Ensure no negative Amount values in revenue_df
revenue_df[revenue_df['Amount'] < 0].shape

(0, 23)

In [ ]:
# Check the number of unique months in the data
revenue_df['Order Month Number'].nunique()

4

In [ ]:
# Save the revenue_df to a CSV file
revenue_df.to_csv('revenue_data.csv', index=False)
# Save the cleaned df1 to a CSV file
df1.to_csv('cleaned_data.csv', index=False)